# gRPC

In [ ]:
! python -m pip install grpcio
! pip install grpcio-tools


## Chatbot

### proto

In [ ]:
import os
if not os.path.isdir('protos'): os.makedirs('protos')

In [ ]:
%%writefile protos/chatbot.proto
syntax = "proto3";

package chatPack;

service ChatService {
  rpc ChatFunc (ChatRequest) returns (ChatReply) {}
}

message ChatRequest {
  string name = 1;
}
message ChatReply {
  string result = 1;
}

In [ ]:
! python -m grpc_tools.protoc -I protos/ --python_out=protos/ --grpc_python_out=protos/ chatbot.proto

### Server

In [ ]:
import os
if not os.path.isdir('chatbot'): os.makedirs('chatbot')

In [22]:
%%writefile chatbot/server.py
from concurrent import futures
import logging

import grpc
import sys
sys.path.append("protos")
import chatbot_pb2
import chatbot_pb2_grpc
sys.path.append("configs")
import config

class ChatService(chatbot_pb2_grpc.ChatService):

    def ChatFunc(self, request, context):
        return chatbot_pb2.ChatReply(result='Chatbot:%s' % request.name)

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    chatbot_pb2_grpc.add_ChatServiceServicer_to_server(ChatService(), server)
    server.add_insecure_port('[::]:'+str(config.chatbot))
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    logging.basicConfig()
    serve()

Overwriting chatbot/server.py


### Start chatbot server
python chatbot/server.py

### Client 

In [32]:
import grpc
import sys
sys.path.append("protos")
import chatbot_pb2
import chatbot_pb2_grpc
sys.path.append("configs")
import config

def chatBotClient():
    with grpc.insecure_channel('localhost:'+str(config.chatbot)) as channel:
        stub = chatbot_pb2_grpc.ChatServiceStub(channel)
        response = stub.ChatFunc(chatbot_pb2.ChatRequest(name='blue'))
    print("Chatbot client received: " + response.result)
chatBotClient()

Chatbot client received: Vaja client received: Vaja:blue


## Vaja

### proto

In [24]:
%%writefile protos/vaja.proto
syntax = "proto3";

package vajaPack;

service VajaService {
  rpc VajaFunc (VajaRequest) returns (VajaReply) {}
  rpc VajaStream (stream StreamData) returns (VajaReply) {}
  rpc VajaBiStream (stream StreamData) returns (stream StreamData) {}
}

message VajaRequest {
  string name = 1;
}
message VajaReply {
  string result = 1;
}
message StreamData {
  string check = 1;
  string data = 2;
}

Overwriting protos/vaja.proto


In [25]:
! python -m grpc_tools.protoc -I protos/ --python_out=protos/ --grpc_python_out=protos/ vaja.proto

### Server

In [ ]:
import os
if not os.path.isdir('vaja'): os.makedirs('vaja')

In [26]:
%%writefile vaja/server.py
from concurrent import futures
import logging

import grpc
import sys
sys.path.append("protos")
import vaja_pb2
import vaja_pb2_grpc
sys.path.append("configs")
import config

class VajaService(vaja_pb2_grpc.VajaService):

    def VajaFunc(self, request, context):
        print('Recive data:'+request.name)
        return vaja_pb2.VajaReply(result='Vaja:%s' % request.name)

    def VajaStream(self, request_iterator, context):
        for new_data in request_iterator:
            print('Stream Recive data:'+new_data.check+':'+new_data.data)
        print('End Stream.')
        return vaja_pb2.VajaReply(result='Vaja:stream')

    def VajaBiStream(self, request_iterator, context):
        for new_data in request_iterator:
            print('BiStream Recive data:'+new_data.check+':'+new_data.data)
            yield vaja_pb2.StreamData(check='1',data=new_data.data+' ok')
        print('End BiStream.')

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    vaja_pb2_grpc.add_VajaServiceServicer_to_server(VajaService(), server)
    server.add_insecure_port('[::]:'+str(config.vaja))
    server.start()
    server.wait_for_termination()

if __name__ == '__main__':
    logging.basicConfig()
    serve()

Overwriting vaja/server.py


### Start chatbot server
python vaja/server.py

### Client 

#### VajaFunc client

In [27]:
import grpc
import sys
sys.path.append("protos")
import vaja_pb2
import vaja_pb2_grpc
sys.path.append("configs")
import config

def vajaClient():
    with grpc.insecure_channel('localhost:'+str(config.vaja)) as channel:
        stub = vaja_pb2_grpc.VajaServiceStub(channel)
        response = stub.VajaFunc(vaja_pb2.VajaRequest(name='blue'))
        print("Vaja client received: " + response.result)
vajaClient()

Vaja client received: Vaja:blue


#### Stream client

In [28]:
import grpc
import sys
sys.path.append("protos")
import vaja_pb2
import vaja_pb2_grpc
sys.path.append("configs")
import config
def make_data(check, data):
    return vaja_pb2.StreamData(
        check=check,
        data=data)
def generate_messages():
    messages = [
        make_data('0','zero'),
        make_data('0','one'),
        make_data('0','two'),
    ]
    for msg in messages:
        yield msg
def streamClient(stub):
    response = stub.VajaStream(generate_messages())
    print("Vaja stream client received: " + response.result)
def biStreamClient(stub): 
    response = stub.VajaBiStream(generate_messages())
    for resp in response:
        print("Vaja biStream client received: " + str(resp.check) + ':' + resp.data)
def run():
    with grpc.insecure_channel('localhost:'+str(config.vaja)) as channel:
        stub = vaja_pb2_grpc.VajaServiceStub(channel)
        streamClient(stub)
        biStreamClient(stub)
run()

Vaja stream client received: Vaja:stream
Vaja biStream client received: 1:zero ok
Vaja biStream client received: 1:one ok
Vaja biStream client received: 1:two ok


## Vaja to Chatbot

In [29]:
%%writefile chatbot/vajaClient.py
import grpc
import sys
sys.path.append("protos")
import vaja_pb2
import vaja_pb2_grpc
sys.path.append("configs")
import config

def vajaClient(nameInput):
    with grpc.insecure_channel('localhost:'+str(config.vaja)) as channel:
        stub = vaja_pb2_grpc.VajaServiceStub(channel)
        response = stub.VajaFunc(vaja_pb2.VajaRequest(name=nameInput))
    return "Vaja client received: " + response.result

Overwriting chatbot/vajaClient.py


In [30]:
%%writefile chatbot/server.py
from concurrent import futures
import logging

import grpc
import sys
sys.path.append("protos")
import chatbot_pb2
import chatbot_pb2_grpc
sys.path.append("configs")
import config

import vajaClient

class ChatService(chatbot_pb2_grpc.ChatService):

    def ChatFunc(self, request, context):
        return chatbot_pb2.ChatReply(result=vajaClient.vajaClient(request.name))

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    chatbot_pb2_grpc.add_ChatServiceServicer_to_server(ChatService(), server)
    server.add_insecure_port('[::]:'+str(config.chatbot))
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    logging.basicConfig()
    serve()

Overwriting chatbot/server.py


## Patii
???

# FastAPI

In [ ]:
! pip install -r FastApi/requirements.txt

## Start server
uvicorn main:app --reload --port 8080

## Test API

In [ ]:
! pip install requests

### Get method

In [33]:
import requests
response = requests.get('http://localhost:8080/?name=blue')
print(response.status_code)
print(response.json())

200
{'Hello': 'blue'}


### Post method

In [34]:
import requests
url = 'http://localhost:8080/'
payload = {
      'name':'blue',
      'age':38
      }

response = requests.post(url, data = payload)
print(response.status_code)
print(response.json())

200
{'Hello': 'blue:38 year old.'}


### json input

In [35]:
import requests
url = 'http://localhost:8080/json'
payload = {
      'name':'blue',
      'age':38
      }

response = requests.post(url, json = payload)
print(response.status_code)
print(response.json())

200
{'Hello': 'blue:38 year old.'}


### items

In [36]:
import requests
response = requests.get('http://localhost:8080/items/26?q=blue')
print(response.status_code)
print(response.json())

200
{'item_id': 26, 'q': 'blue'}


### gRPC Client

In [37]:
import requests
response = requests.get('http://localhost:8080/route?name=blue')
print(response.status_code)
print(response.json())

200
{'Route to': 'blue', 'ret': 'Chat client received: Vaja client received: Vaja:blue'}


#### json input
???